In [1]:
import numpy as np
import json
import random
import importlib
from pprint import pprint

import os
import torch
from tbsim.configs.scene_edit_config import SceneEditingConfig

In [2]:
# args & configs
cfg = SceneEditingConfig()
config_file = '/local-scratch2/lufan/traceMR/configs/eval/orca/target_pos.json'
eval_class = 'Diffuser'
policy_ckpt_dir = '/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed'
policy_ckpt_key = 'iter40000'
results_root_dir = '/local-scratch2/lufan/traceMR/out/orca_mixed_out'
num_scenes_per_batch = 1
render = False
render_img = True
render_size = 400
render_px_per_m = 12.0
seed = None
prefix = None
ground_truth = False
editing_source = 'heuristic'

if config_file is not None:
    external_cfg = json.load(open(config_file, "r"))
    cfg.update(**external_cfg)

if eval_class is not None:
    cfg.eval_class = eval_class

if policy_ckpt_dir is not None:
    assert policy_ckpt_key is not None, "Please specify a key to look for the checkpoint, e.g., 'iter50000'"
    cfg.ckpt.policy.ckpt_dir = policy_ckpt_dir
    cfg.ckpt.policy.ckpt_key = policy_ckpt_key

if num_scenes_per_batch is not None:
    cfg.num_scenes_per_batch = num_scenes_per_batch

if cfg.name is None:
    cfg.name = cfg.eval_class

if prefix is not None:
    cfg.name = prefix + cfg.name

if seed is not None:
    cfg.seed = seed
if results_root_dir is not None:
    cfg.results_dir = os.path.join(results_root_dir, cfg.name)
else:
    cfg.results_dir = os.path.join(cfg.results_dir, cfg.name)
assert cfg.env == "trajdata", "trajdata is the only supported environment"
if editing_source is not None:
    cfg.edits.editing_source = editing_source
if not isinstance(cfg.edits.editing_source, list):
    cfg.edits.editing_source = [cfg.edits.editing_source]

cfg.experience_hdf5_path = os.path.join(cfg.results_dir, "data.hdf5")

for k in cfg[cfg.env]:  # copy env-specific config to the global-level
    cfg[k] = cfg[cfg.env][k]
cfg.pop("trajdata")

render_cfg = {
    'size' : render_size,
    'px_per_m' : render_px_per_m,
}

cfg.lock()

In [3]:
# basic setup, before init policy
from tbsim.utils.trajdata_utils import set_global_trajdata_batch_env
"""
run_scene_editor(
    cfg,
    save_cfg=True,
    data_to_disk=True,
    render_to_video=args.render,
    render_to_img=args.render_img,
    render_cfg=render_cfg,
    use_gt=args.ground_truth,
)
"""
eval_cfg = cfg
save_cfg = True
data_to_disk = True
render_to_video = render
render_to_img = render_img
render_cfg = render_cfg
use_gt = ground_truth

set_global_trajdata_batch_env(eval_cfg.trajdata_source_test[0])
print(eval_cfg)
# for reproducibility
np.random.seed(eval_cfg.seed)
random.seed(eval_cfg.seed)
torch.manual_seed(eval_cfg.seed)
torch.cuda.manual_seed(eval_cfg.seed)

# basic setup
print('saving results to {}'.format(eval_cfg.results_dir))
os.makedirs(eval_cfg.results_dir, exist_ok=True)
if render_to_video:
    os.makedirs(os.path.join(eval_cfg.results_dir, "videos/"), exist_ok=True)
if render_to_video or render_to_img:
    os.makedirs(os.path.join(eval_cfg.results_dir, "viz/"), exist_ok=True)
if save_cfg:
    json.dump(eval_cfg, open(os.path.join(eval_cfg.results_dir, "config.json"), "w+"))
if data_to_disk and os.path.exists(eval_cfg.experience_hdf5_path):
    os.remove(eval_cfg.experience_hdf5_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


{
    "name": "orca_map_open_loop_target_pos",
    "env": "trajdata",
    "dataset_path": null,
    "eval_class": "Diffuser",
    "seed": 0,
    "num_scenes_per_batch": 1,
    "num_scenes_to_evaluate": 100,
    "num_episode_repeats": 1,
    "start_frame_index_each_episode": null,
    "seed_each_episode": null,
    "ego_only": false,
    "ckpt_root_dir": "checkpoints/",
    "experience_hdf5_path": "/local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos/data.hdf5",
    "results_dir": "/local-scratch2/lufan/traceMR/out/orca_mixed_out/orca_map_open_loop_target_pos",
    "ckpt": {
        "policy": {
            "ckpt_dir": "/local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed",
            "ckpt_key": "iter40000",
            "ngc_job_id": null
        }
    },
    "policy": {
        "num_action_samples": 20,
        "guide_as_filter_only": false,
        "guide_with_gt": false,
        "class_free_guide_w": 0.0,
        "guide_clean": true
    },
    "metrics": {
 

In [5]:
# create policy and rollout wrapper
policy_composers = importlib.import_module("tbsim.evaluation.policy_composers")
composer_class = getattr(policy_composers, eval_cfg.eval_class)
composer = composer_class(eval_cfg, device)
policy, exp_config = composer.get_policy()
policy_model = policy.model

Checkpoint path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/iter40000.ckpt
Config path: /local-scratch2/lufan/traceMR/ckpt/trace/orca_mixed/config.json
DIFFUSER: Dropping map input conditioning with p = 0.100000 during training...
DIFFUSER: Dropping neighbor traj input conditioning with p = 0.100000 during training...
[ models/temporal ] Channel dimensions: [(38, 64), (64, 128), (128, 256)]
> /local-scratch2/lufan/traceMR/tbsim/algos/algos.py(89)__init__()
     87         # set up initial guidance
     88         if guidance_config is not None:
---> 89             self.set_guidance(guidance_config)
     90 
     91         # set up EMA

